In [1]:
import math
import os
import openpyxl
import platform
import numpy as np
import pandas as pd

In [2]:
if platform.platform()[:3].lower() == 'mac':
    mac_m1 = True
elif platform.platform()[:3].lower() == 'win':
    mac_m1 = False
    
if mac_m1:
    path = '/Users/rainyseason/winston/Workspace/python/Pycharm Project/sinewave_analyze/Evaluation/'
    path_csv = path + 'tek_csv/'
    path_excel = path + 'tek_excel/'
    path_summary = path + 'summary/'
    path_information = path + 'test information/'
    path_kmon = path + 'kmon_csv'
else:
    path = 'D:/data_analyze/'
    path_csv = path + 'tek_csv/'
    path_excel = path + 'tek_excel/'
    path_summary = path + 'summary/'
    path_information = path + 'test information/'
    path_kmon = path + 'kmon_csv/'
    
evaluation_control_file = 'eval_control.xlsx'

In [4]:
files = os.listdir(path_excel)
files = [file for file in files if '.DS' not in file and '~' not in file]
files.sort()
for file in enumerate(files):
    print(file)

(0, 'Ch3 Ch4')
(1, 'tek0000.xlsx')
(2, 'tek00000.xlsx')
(3, 'tek00001.xlsx')
(4, 'tek00002.xlsx')
(5, 'tek00003.xlsx')
(6, 'tek00004.xlsx')
(7, 'tek00005.xlsx')
(8, 'tek00006.xlsx')
(9, 'tek00007.xlsx')
(10, 'tek00008.xlsx')
(11, 'tek00009.xlsx')
(12, 'tek0001.xlsx')
(13, 'tek00010.xlsx')
(14, 'tek00011.xlsx')
(15, 'tek00012.xlsx')
(16, 'tek00013.xlsx')
(17, 'tek00014.xlsx')
(18, 'tek00015.xlsx')
(19, 'tek00016.xlsx')
(20, 'tek00017.xlsx')
(21, 'tek00018.xlsx')
(22, 'tek00019.xlsx')
(23, 'tek00020.xlsx')
(24, 'tek00021.xlsx')
(25, 'tek00022.xlsx')
(26, 'tek00023.xlsx')
(27, 'tek00024.xlsx')
(28, 'tek00025.xlsx')
(29, 'tek00026.xlsx')
(30, 'tek00027.xlsx')
(31, 'tek00028.xlsx')
(32, 'tek00029.xlsx')
(33, 'tek00030.xlsx')
(34, 'tek00031.xlsx')
(35, 'tek00032.xlsx')
(36, 'tek00033.xlsx')
(37, 'tek00034.xlsx')
(38, 'tek00035.xlsx')
(39, 'tek00036.xlsx')
(40, 'tek00037.xlsx')
(41, 'tek00038.xlsx')


In [9]:
del files[0]

In [11]:
for file in files:
    src = path_excel + file
    name = file.split('.')[0]
    extension = file[-5:]
    print(src, dst)

D:/data_analyze/tek_excel/tek0000.xlsx D:/data_analyze/tek_excel/tek00038 08Ch3xlsx
D:/data_analyze/tek_excel/tek00000.xlsx D:/data_analyze/tek_excel/tek00038 08Ch3xlsx
D:/data_analyze/tek_excel/tek00001.xlsx D:/data_analyze/tek_excel/tek00038 08Ch3xlsx
D:/data_analyze/tek_excel/tek00002.xlsx D:/data_analyze/tek_excel/tek00038 08Ch3xlsx
D:/data_analyze/tek_excel/tek00003.xlsx D:/data_analyze/tek_excel/tek00038 08Ch3xlsx
D:/data_analyze/tek_excel/tek00004.xlsx D:/data_analyze/tek_excel/tek00038 08Ch3xlsx
D:/data_analyze/tek_excel/tek00005.xlsx D:/data_analyze/tek_excel/tek00038 08Ch3xlsx
D:/data_analyze/tek_excel/tek00006.xlsx D:/data_analyze/tek_excel/tek00038 08Ch3xlsx
D:/data_analyze/tek_excel/tek00007.xlsx D:/data_analyze/tek_excel/tek00038 08Ch3xlsx
D:/data_analyze/tek_excel/tek00008.xlsx D:/data_analyze/tek_excel/tek00038 08Ch3xlsx
D:/data_analyze/tek_excel/tek00009.xlsx D:/data_analyze/tek_excel/tek00038 08Ch3xlsx
D:/data_analyze/tek_excel/tek0001.xlsx D:/data_analyze/tek_excel/t

In [18]:
path_ch3 = path_excel
path_ch4 = path + 'Ch3 Ch4/'

In [8]:
file

'Ch3 Ch4'